In [21]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
        def __init__(self):
            super(Net, self).__init__()
            # 1 input image channel, 6 output channels, 5x5 square conv
            self.conv1 = nn.Conv2d(1,6,5)
            self.conv2 = nn.Conv2d(6,16,5)
            # an affine operation: y= Wx +b
            self.fc1 = nn.Linear(16 * 5 * 5,120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)
            
        def forward(self, x):
            #max pooling over a (2,2) window
            x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
            # if the size sis a square you can only specify a single number
            x = F.max_pool2d(F.relu(self.conv2(x)), 2)
            x = x.view(-1, self.num_flat_features(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x
        
        def num_flat_features(self, x):
            size = x.size()[1:] # all dimensions except the batch dimension
            num_features = 1
            for s in size:
                num_features *= s
            return num_features
        
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [22]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [31]:
input = Variable(torch.randn(1,1,32,32))

In [32]:
out = net(input)
print(out)

Variable containing:
 0.0706  0.1230 -0.1728  0.1408  0.0243 -0.1765 -0.0866  0.0016  0.0205 -0.0352
[torch.FloatTensor of size 1x10]



In [33]:
net.zero_grad()

In [35]:
out.backward(torch.rand(1,10))

In [48]:
output = net(input)
target = Variable(torch.arange(1,11))
criterion = nn.MSELoss()

In [49]:
loss = criterion(output, target)
print(loss)

Variable containing:
 38.7786
[torch.FloatTensor of size 1]



In [61]:
print(loss.creator)  # MSELoss
print(loss.creator.previous_functions[0][0])  # Linear


In [62]:
net.zero_grad()
print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.1025
-0.0254
 0.0055
 0.0435
 0.1463
 0.0890
[torch.FloatTensor of size 6]



In [63]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)


In [64]:
import torch.optim as optim

#create your optimizer
optimizer = optim.SGD(net.parameters(), lr=lr)

# in your training loop:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()